In [2]:
import torch
import torchvision

In [3]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [4]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

9920512it [00:09, 996423.19it/s]                                                                                       


Extracting /files/MNIST\raw\train-images-idx3-ubyte.gz to /files/MNIST\raw


32768it [00:00, 77648.29it/s]                                                                                          


Extracting /files/MNIST\raw\train-labels-idx1-ubyte.gz to /files/MNIST\raw


1654784it [00:04, 333989.40it/s]                                                                                       


Extracting /files/MNIST\raw\t10k-images-idx3-ubyte.gz to /files/MNIST\raw


8192it [00:00, 26112.78it/s]                                                                                           


Extracting /files/MNIST\raw\t10k-labels-idx1-ubyte.gz to /files/MNIST\raw
Processing...
Done!


In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [7]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [10]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [13]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), 'model.pth')
            torch.save(optimizer.state_dict(), 'optimizer.pth')

def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset),
          100. * correct / len(test_loader.dataset)))

In [14]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

c:\users\dmitriy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. loss: 2.3284, Accuracy: 1141/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.385978
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.290516
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.253946
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.257330
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.225238
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.233637
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.270044
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.207447
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.092492
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.045552
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.996288
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.768286
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.015460
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.652786
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.716916
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.523116
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.364895
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.504792
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.379674
Train Epoch: 1 [12160

Train Epoch: 2 [43520/60000 (72%)]	Loss: 0.246361
Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.522963
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.423486
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.397106
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.519900
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.301361
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.369362
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.506380
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.332788
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.154262
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.351759
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.246596
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.527776
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.419197
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.218135
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.311199
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.325079
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.311598
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.448082
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.343415
